In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

## 이번에는 영화 리뷰를 기반으로 감성 분석을 진행해볼 것이다
 사용되어질 데이터는 IMDB 데이터셋으로 train 2.5만개, test 2.5만개로 구성되어 있으며 긍적적인 리뷰와 부정적인 리뷰의 수는 동일하도록 구성되어 있다.

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file('aclIndb_v1', url, untar =True, cache_dir = '.', cache_subdir='')
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84125825/84125825 [==============================] - 3s 0us/step


In [3]:
os.listdir(dataset_dir)

['imdb.vocab', 'train', 'test', 'README', 'imdbEr.txt']

In [5]:
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

os.listdir(train_dir), os.listdir(test_dir)

(['neg',
  'unsupBow.feat',
  'unsup',
  'urls_neg.txt',
  'urls_pos.txt',
  'labeledBow.feat',
  'urls_unsup.txt',
  'pos'],
 ['neg', 'urls_neg.txt', 'urls_pos.txt', 'labeledBow.feat', 'pos'])

In [9]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())


print(len(os.listdir(os.path.join(train_dir, 'pos'))))

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.
12500


In [13]:
remove_dir = os.path.join(train_dir, 'unsup')
# 아래의 코드는 폴더안에 파일이 있으면 사용할 수 없다.
# os.rmdir(remove_dir)
shutil.rmtree(remove_dir)

In [19]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(train_dir,
                                                          batch_size=batch_size,
                                                          validation_split=0.2,
                                                          subset='training',
                                                          seed = seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [20]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print('review', text_batch.numpy()[i])
    print('label', label_batch.numpy()[i])

review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
label 0
review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [22]:
print('label 0 means -', raw_train_ds.class_names[0])
print('label 1 means -', raw_train_ds.class_names[1])

label 0 means - neg
label 1 means - pos


In [23]:
# 사실 아래의 코드에 subset = 'both' 로 설정하면 한번에 train과 val 데이터셋이 리턴된다.
raw_val_ds = tf.keras.utils.text_dataset_from_directory(train_dir,
                                                        batch_size = batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'validation',
                                                        seed = seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [24]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(test_dir,
                                                         batch_size= batch_size)

Found 25000 files belonging to 2 classes.
